In [1]:
import pandas as pd
import regex as re
import pygsheets
from datetime import date, timedelta, datetime
import numpy as np
import geopandas as gpd

In [129]:
def get_revcircle_damages_dict(ROW):    
    DETAIL = ROW['Details']
    REV_CIRCLE_DAMAGES_DICT = dict()
    
    count = 0
    rev_circles_identified = []
    rev_circles_identifed_at = []
    for REV_CIRCLE in REVENUE_CIRCLES:
        if REV_CIRCLE in DETAIL:
            rev_circles_identified.append(REV_CIRCLE)
            rev_circles_identifed_at.append(DETAIL.index(REV_CIRCLE))
            count = count+1
    REV_CIRCLES_IDENTIFIED_DF = pd.DataFrame([rev_circles_identified,rev_circles_identifed_at]).T
    REV_CIRCLES_IDENTIFIED_DF.columns = ['RevenueCircle','StringIndex']
    REV_CIRCLES_IDENTIFIED_DF = REV_CIRCLES_IDENTIFIED_DF.sort_values(by='StringIndex').reset_index(drop=True)
    
    for i,val in enumerate(REV_CIRCLES_IDENTIFIED_DF.StringIndex):
        StringStartIndex = val
        if i+1 == len(REV_CIRCLES_IDENTIFIED_DF.StringIndex):
            StringEndIndex = None
        else:
            StringEndIndex = REV_CIRCLES_IDENTIFIED_DF.StringIndex[i+1]
        
        SUB_DETAIL = DETAIL[StringStartIndex:StringEndIndex]
        
        if other_damages == True:
            NO_DAMAGES = len(re.findall('\|', SUB_DETAIL))
            print(NO_DAMAGES)
            REV_CIRCLE_DAMAGES_DICT[REV_CIRCLES_IDENTIFIED_DF.RevenueCircle[i].split('-')[0]] = NO_DAMAGES-1
            continue 
            
        NO_DAMAGES_DateAlgo = len(re.findall('2023',SUB_DETAIL)) + len(re.findall(r'\d+/\d+/\d{2}\s+', SUB_DETAIL)) + len(re.findall(r'\d+-\d+-\d{2}\s+', SUB_DETAIL)) + len(re.findall(r'\d+\.\d+\.\d{2}\s+', SUB_DETAIL))
        # Number of 2022 in the DETAIL +  Number of dd/mm/22 in the DETAIL +  Number of dd-mm-22 in the DETAIL + Number of dd.mm.22 in the DETAIL
        NO_DAMAGES_LongAlgo = len(re.findall('Long -',SUB_DETAIL)) 
    
        if float(ROW['Number of Damages (Long)'])==float(ROW['Number']):
            REV_CIRCLE_DAMAGES_DICT[REV_CIRCLES_IDENTIFIED_DF.RevenueCircle[i].split('-')[0]] = NO_DAMAGES_LongAlgo
        elif float(ROW['Number of Damages (Long)'])>float(ROW['Number of Damages (Date)']):
            REV_CIRCLE_DAMAGES_DICT[REV_CIRCLES_IDENTIFIED_DF.RevenueCircle[i].split('-')[0]] = NO_DAMAGES_LongAlgo
        else:
            REV_CIRCLE_DAMAGES_DICT[REV_CIRCLES_IDENTIFIED_DF.RevenueCircle[i].split('-')[0]] = NO_DAMAGES_DateAlgo
    
    
    return REV_CIRCLE_DAMAGES_DICT    

In [118]:
ASSAM_REVENUE_CIRCLES = pd.read_csv('Data/Assam_Maps/ASSAM_REVENUE_CIRCLES_FRIMS_NAMES.csv')
ASSAM_REVENUE_CIRCLES.revenue_ci = ASSAM_REVENUE_CIRCLES.revenue_ci.str.replace(r'\(Pt\)','', regex=True)
ASSAM_REVENUE_CIRCLES.revenue_ci = ASSAM_REVENUE_CIRCLES.revenue_ci.str.replace(r'\(Pt-I\)','', regex=True)
ASSAM_REVENUE_CIRCLES.revenue_ci = ASSAM_REVENUE_CIRCLES.revenue_ci.str.replace(r'\(Pt-II\)','', regex=True)

ASSAM_REVENUE_CIRCLES.revenue_ci = ASSAM_REVENUE_CIRCLES.revenue_ci.str.strip()

# Table of Contents:
* [Roads damaged per revenue circle](#roadsdamaged)
* [Embankment damages per revenue circle](#embankmentdamages)
* [Embankments affected per revenue circle](#embankmentaffected)
* [Bridges damaged per revenue circle](#bridgesdamaged)

## EMBANKMENT AFFECTED <a class="anchor" id="embankmentaffected"></a>

In [154]:
embankments_affected_df = pd.read_csv('Data_2023/Cleaned Data/FRIMS_EMBANKMENTS_AFFECTED_MASTER_2023.csv')


embankments_affected_df['Details'] = embankments_affected_df.Details.str.replace('Baghbar','Baghbor',regex=True)
embankments_affected_df['Details'] = embankments_affected_df.Details.str.replace('Dotma','Dotoma',regex=True)
embankments_affected_df['Details'] = embankments_affected_df.Details.str.replace('Palashbari','Palasbari',regex=True)
embankments_affected_df['Details'] = embankments_affected_df.Details.str.replace('North Ghy','North\nGuwahati',regex=True)
embankments_affected_df['Details'] = embankments_affected_df.Details.str.replace('Khairabari','Khoirabari',regex=True)
embankments_affected_df['Details'] = embankments_affected_df.Details.str.replace('Dangtol','Bongaigaon',regex=True)
embankments_affected_df['Details'] = embankments_affected_df.Details.str.replace('Naduar','Na-Duar',regex=True)


embankments_affected_df['District'] = embankments_affected_df.District.str.replace('Dima-Hasao','Dima Hasao',regex=True)
embankments_affected_df['District'] = embankments_affected_df.District.str.replace('Sivasagar','Sivsagar',regex=True)


embankments_affected_df.tail()

,Date,District,Number,Details
31,2023-06-24,Nalbari,10,Nalbari - Pagaldia embankment | Bhadra | As pe...
32,2023-06-24,Kamrup,3,Hajo - Puthimari embankment | Hadal manakuchi ...
33,2023-06-24,Golaghat,1,Bokakhat - Un embankment portion of Alengmari ...
34,2023-06-24,Biswanath,1,Gohpur - Mild bank erosion is observed at R/Ba...
35,2023-06-25,Nalbari,7,Nalbari - Pagaldia embankment RB | Moiradanga ...


In [155]:
global REVENUE_CIRCLES
global other_damages
other_damages = False

FILTERED_DAMAGES_dfs = []

for DISTRICT in embankments_affected_df.District.unique():
    print(DISTRICT)
    FILTERED_DAMAGES_df = embankments_affected_df[embankments_affected_df.District == DISTRICT]
    FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.sort_values(by=['Detail_Length'])
    
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.drop_duplicates(['District','Number','Date'],keep='last')
    
    FILTERED_DAMAGES_df['Number of Damages (Date)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('2023', x))+len(re.findall(r'\d+/\d+/\d{2}\s+',x))+len(re.findall(r'\d+-\d+-\d{2}\s+',x))+len(re.findall(r'\d+\.\d+\.\d{2}\s+', x)))
    FILTERED_DAMAGES_df['Number of Damages (Long)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('Long -', x)))
        
    
    REVENUE_CIRCLES = [REV_CIRCLE.strip()+' -' for REV_CIRCLE in list(ASSAM_REVENUE_CIRCLES[ASSAM_REVENUE_CIRCLES.district_35==DISTRICT.upper()].revenue_ci)]
    print(REVENUE_CIRCLES)
    print('----')
    FILTERED_DAMAGES_df['NumberofDamages_RevenueCircle_level'] = FILTERED_DAMAGES_df.apply(get_revcircle_damages_dict, axis=1)
    FILTERED_DAMAGES_dfs.append(FILTERED_DAMAGES_df)

Dhemaji
['Dhakuakhana -', 'Dhemaji -', 'Gogamukh -', 'Jonai -', 'Sissiborgaon -', 'Subansiri -']
----
Biswanath
['Biswanath -', 'Gohpur -', 'Halem -']
----
Darrang
['Dalgaon -', 'Khoirabari -', 'Mangaldoi -', 'Patharighat -', 'Sipajhar -']
----
Tamulpur
['Bajali -', 'Goreswar -', 'Pathorighat -', 'Rangia -', 'Tamulpur -']
----
Golaghat
['Bokakhat -', 'Dergaon -', 'Golaghat -', 'Khumtai -', 'Morangi -', 'Sarupathar -']
----
Goalpara
['Balijana -', 'Dudhnoi -', 'Lakhipur -', 'Matia -', 'Rangjuli -']
----
Karimganj
['Badarpur -', 'Karimganj -', 'Nilambazar -', 'Patherkandi -', 'RK Nagar -']
----
Lakhimpur
['Bihpuria -', 'Dhakuakhana -', 'Kadam -', 'Narayanpur -', 'North Lakhimpur -', 'Nowboicha -', 'Subansiri -']
----
Nagaon
['Dhing -', 'Kaliabor -', 'Kampur -', 'Nagaon -', 'Raha -', 'Rupahi -', 'Samaguri -']
----
Nalbari
['Baganpara -', 'Banekuchi -', 'Barbhag -', 'Barkhetri -', 'Ghograpar -', 'Nalbari -', 'Pachim\nNalbari -', 'Tihu -']
----
Bajali
['Bajali -', 'Jalah -', 'Sarupeta -']
-

/tmp/ipykernel_5155/797364889.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))


In [156]:
MASTER_DAMAGES = pd.concat(FILTERED_DAMAGES_dfs).reset_index(drop=True)

In [157]:
MASTER_DAMAGES

,Date,District,Number,Details,Detail_Length,Number of Damages (Date),Number of Damages (Long),NumberofDamages_RevenueCircle_level
0,2023-06-17,Dhemaji,1,Jonai - Water Resource Deptt. river side and C...,361,1,1,{'Jonai ': 1}
1,2023-06-14,Dhemaji,1,Gogamukh - Na-Nadi Right Bank Embankment from ...,406,1,1,{'Gogamukh ': 1}
2,2023-06-15,Dhemaji,1,Dhemaji - Kumatiya left Embankment from NH-15 ...,408,1,1,{'Dhemaji ': 1}
3,2023-06-11,Dhemaji,1,Sissiborgaon - Brahmaputra Dyke from Deorighat...,423,1,1,{'Sissiborgaon ': 1}
4,2023-06-24,Biswanath,1,Gohpur - Mild bank erosion is observed at R/Ba...,172,1,1,{'Gohpur ': 1}
5,2023-06-21,Biswanath,1,Gohpur - R/Bank embankment of Sessa | at Augur...,178,1,1,{'Gohpur ': 1}
6,2023-06-20,Biswanath,1,Gohpur - Mild bark erosion is observed R/Bank ...,187,1,1,{'Gohpur ': 1}
7,2023-06-19,Biswanath,1,Gohpur - Dakhin Deona Bari R/Bank embankment o...,248,1,1,{'Gohpur ': 1}
8,2023-06-22,Biswanath,2,Gohpur - Mild Bank Erosion is observed at R/Ba...,359,2,2,{'Gohpur ': 2}
9,2023-06-23,Biswanath,2,Gohpur - Mild Bank Erosion is observed at R/Ba...,359,2,2,{'Gohpur ': 2}


In [134]:
embankment_affected_revcircle_df = pd.DataFrame(columns=['District','RevenueCircle','Number of damages','Date'])
i = 0
for idx, row in MASTER_DAMAGES.iterrows():
    damages_dict = row['NumberofDamages_RevenueCircle_level']
    for key in damages_dict.keys():
        rev_circle = key
        no_damages = damages_dict[key]
        
        embankment_affected_revcircle_df.loc[i,'RevenueCircle'] = rev_circle
        embankment_affected_revcircle_df.loc[i,'District'] = row['District']
        embankment_affected_revcircle_df.loc[i,'Number of damages'] = no_damages
        embankment_affected_revcircle_df.loc[i,'Date'] = row['Date']
        i=i+1
embankment_affected_revcircle_df.Date = pd.to_datetime(embankment_affected_revcircle_df.Date)

In [135]:
embankment_affected_revcircle_df.to_csv('Data_2023/Cleaned Data/RC_FRIMS_EMBANKMENTS_AFFECTED_MASTER_2023.csv', index=False)

## BRIDGES DAMAGED <a class="anchor" id="bridgesdamaged"></a>

In [136]:
bridges_df = pd.read_csv('Data_2023/Cleaned Data/FRIMS_BRIDGES_DAMAGED_MASTER_2023.csv')


bridges_df['Details'] = bridges_df.Details.str.replace('Baghbar','Baghbor',regex=True)
bridges_df['Details'] = bridges_df.Details.str.replace('Dotma','Dotoma',regex=True)
bridges_df['Details'] = bridges_df.Details.str.replace('Palashbari','Palasbari',regex=True)
bridges_df['Details'] = bridges_df.Details.str.replace('North Ghy','North\nGuwahati',regex=True)
bridges_df['Details'] = bridges_df.Details.str.replace('Khairabari','Khoirabari',regex=True)
bridges_df['Details'] = bridges_df.Details.str.replace('Dangtol','Bongaigaon',regex=True)
bridges_df['Details'] = bridges_df.Details.str.replace('Naduar','Na-Duar',regex=True)


bridges_df['District'] = bridges_df.District.str.replace('Dima-Hasao','Dima Hasao',regex=True)


bridges_df.tail()

,Date,District,Number,Details
19,2023-06-23,Barpeta,3,Baghbor - 1. RCC BR No. 7/1 (Barpeta Baghbor r...
20,2023-06-23,Bajali,2,Sarupeta - RCC Br. No. 2/1 | Sarupeta Gaon | R...
21,2023-06-24,Dhubri,1,Dhubri - APPROACHES OF SPT BR. NO. 3/1 AND ROA...
22,2023-06-25,Bongaigaon,2,Srijangram - Golapara to Kirtanpara road | Gol...
23,2023-06-25,Kokrajhar,1,Dotoma - Rabhapara ward no.4 to Ward no5 conne...


In [137]:
global REVENUE_CIRCLES
global other_damages
other_damages = False

FILTERED_DAMAGES_dfs = []

for DISTRICT in bridges_df.District.unique():
    print(DISTRICT)
    FILTERED_DAMAGES_df = bridges_df[bridges_df.District == DISTRICT]
    FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.sort_values(by=['Detail_Length'])
    
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.drop_duplicates(['District','Number','Date'],keep='last')
    
    FILTERED_DAMAGES_df['Number of Damages (Date)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('2022', x))+len(re.findall(r'\d+/\d+/\d{2}\s+',x))+len(re.findall(r'\d+-\d+-\d{2}\s+',x))+len(re.findall(r'\d+\.\d+\.\d{2}\s+', x)))
    FILTERED_DAMAGES_df['Number of Damages (Long)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('Long -', x)))
        
    
    REVENUE_CIRCLES = [REV_CIRCLE.strip()+' -' for REV_CIRCLE in list(ASSAM_REVENUE_CIRCLES[ASSAM_REVENUE_CIRCLES.district_35==DISTRICT.upper()].revenue_ci)]
    print(REVENUE_CIRCLES)
    print('----')
    FILTERED_DAMAGES_df['NumberofDamages_RevenueCircle_level'] = FILTERED_DAMAGES_df.apply(get_revcircle_damages_dict, axis=1)
    FILTERED_DAMAGES_dfs.append(FILTERED_DAMAGES_df)

Dhemaji
['Dhakuakhana -', 'Dhemaji -', 'Gogamukh -', 'Jonai -', 'Sissiborgaon -', 'Subansiri -']
----
Sonitpur
['Chariduar -', 'Dhekiajuli -', 'Na-Duar -', 'Tezpur -']
----
Darrang
['Dalgaon -', 'Khoirabari -', 'Mangaldoi -', 'Patharighat -', 'Sipajhar -']
----
Kokrajhar
['Bagribari -', 'Bhawraguri -', 'Chapar -', 'Dhubri -', 'Dotoma -', 'Golokganj -', 'Gossaigaon -', 'Kokrajhar -']
----
Barpeta
['Baghbor -', 'Barnagar -', 'Barpeta -', 'Chenga -', 'Kalgachia -', 'Sarthebari -']
----
Bongaigaon
['Boitamari -', 'Bongaigaon -', 'Manikpur -', 'Sidli -', 'Srijangram -']
----
Baksa
['Baganpara -', 'Barama -', 'Barnagar -', 'Baska -', 'Ghograpar -', 'Jalah -', 'Sarupeta -']
----
Nalbari
['Baganpara -', 'Banekuchi -', 'Barbhag -', 'Barkhetri -', 'Ghograpar -', 'Nalbari -', 'Pachim\nNalbari -', 'Tihu -']
----
Dhubri
['Agamoni -', 'Bagribari -', 'Bilasipara -', 'Chapar -', 'Dhubri -', 'Golakganj -', 'Gossaigaon -']
----
Biswanath
['Biswanath -', 'Gohpur -', 'Halem -']
----
Bajali
['Bajali -', 'J

/tmp/ipykernel_5155/1577161483.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))


In [138]:
MASTER_DAMAGES = pd.concat(FILTERED_DAMAGES_dfs).reset_index(drop=True)

In [139]:
bridges_revcircle_df = pd.DataFrame(columns=['District','RevenueCircle','Number of damages','Date'])
i = 0
for idx, row in MASTER_DAMAGES.iterrows():
    damages_dict = row['NumberofDamages_RevenueCircle_level']
    for key in damages_dict.keys():
        rev_circle = key
        no_damages = damages_dict[key]
        
        bridges_revcircle_df.loc[i,'RevenueCircle'] = rev_circle
        bridges_revcircle_df.loc[i,'District'] = row['District']
        bridges_revcircle_df.loc[i,'Number of damages'] = no_damages
        bridges_revcircle_df.loc[i,'Date'] = row['Date']
        i=i+1
bridges_revcircle_df.Date = pd.to_datetime(bridges_revcircle_df.Date)

In [140]:
bridges_revcircle_df.RevenueCircle = bridges_revcircle_df.RevenueCircle.str.replace('Na','Na-Duar')

In [141]:
bridges_revcircle_df.to_csv('Data_2023/Cleaned Data/RC_FRIMS_BRIDGES_DAMAGED_MASTER_2023.csv', index=False)

## ROADS DAMAGED <a class="anchor" id="roadsdamaged"></a>

In [142]:
roads_damaged_df = pd.read_csv('Data_2023/Cleaned Data/FRIMS_ROADS_DAMAGED_MASTER_2023.csv')

roads_damaged_df['Details'] = roads_damaged_df.Details.str.replace('Baghbar','Baghbor',regex=True)
roads_damaged_df['Details'] = roads_damaged_df.Details.str.replace('Dotma','Dotoma',regex=True)
roads_damaged_df['Details'] = roads_damaged_df.Details.str.replace('Palashbari','Palasbari',regex=True)
roads_damaged_df['Details'] = roads_damaged_df.Details.str.replace('North Ghy','North\nGuwahati',regex=True)
roads_damaged_df['Details'] = roads_damaged_df.Details.str.replace('Khairabari','Khoirabari',regex=True)
roads_damaged_df['Details'] = roads_damaged_df.Details.str.replace('Dangtol','Bongaigaon',regex=True)
roads_damaged_df['Details'] = roads_damaged_df.Details.str.replace('Naduar','Na-Duar',regex=True)


roads_damaged_df['District'] = roads_damaged_df.District.str.replace('Dima-Hasao','Dima Hasao',regex=True)


roads_damaged_df.head()

,Date,District,Number,Details
0,2023-06-11,Biswanath,1,Gohpur - Singarajan to Sutardoloni Kutcha Road...
1,2023-06-12,Biswanath,6,Gohpur - Road from Borigaon to Deonabori. (A c...
2,2023-06-12,Darrang,1,Mangaldoi - Approach road Noanadi Kachia Bund ...
3,2023-06-13,Lakhimpur,1,Narayanpur - Balikuchi Bishnupur Road at Ch 35...
4,2023-06-14,Dhemaji,2,Dhemaji - PWD Deptt. Road from Jiadhal charial...


In [143]:
roads_damaged_df = roads_damaged_df.sort_values(by='Date')

In [144]:
global REVENUE_CIRCLES
global other_damages
other_damages = False

FILTERED_DAMAGES_dfs = []

for DISTRICT in roads_damaged_df.District.unique():
    print(DISTRICT)
    FILTERED_DAMAGES_df = roads_damaged_df[roads_damaged_df.District == DISTRICT]
    FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.sort_values(by=['Detail_Length'])
    
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.drop_duplicates(['District','Number','Date'],keep='last')
    
    FILTERED_DAMAGES_df['Number of Damages (Date)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('2023', x))+len(re.findall(r'\d+/\d+/\d{2}\s+',x))+len(re.findall(r'\d+-\d+-\d{2}\s+',x))+len(re.findall(r'\d+\.\d+\.\d{2}\s+', x)))
    FILTERED_DAMAGES_df['Number of Damages (Long)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('Long -', x)))
        
    
    REVENUE_CIRCLES = [REV_CIRCLE.strip()+' -' for REV_CIRCLE in list(ASSAM_REVENUE_CIRCLES[ASSAM_REVENUE_CIRCLES.district_35==DISTRICT.upper()].revenue_ci)]
    print(REVENUE_CIRCLES)
    print('----')
    FILTERED_DAMAGES_df['NumberofDamages_RevenueCircle_level'] = FILTERED_DAMAGES_df.apply(get_revcircle_damages_dict, axis=1)
    FILTERED_DAMAGES_dfs.append(FILTERED_DAMAGES_df)

Biswanath
['Biswanath -', 'Gohpur -', 'Halem -']
----
Darrang
['Dalgaon -', 'Khoirabari -', 'Mangaldoi -', 'Patharighat -', 'Sipajhar -']
----
Lakhimpur
['Bihpuria -', 'Dhakuakhana -', 'Kadam -', 'Narayanpur -', 'North Lakhimpur -', 'Nowboicha -', 'Subansiri -']
----
Dhemaji
['Dhakuakhana -', 'Dhemaji -', 'Gogamukh -', 'Jonai -', 'Sissiborgaon -', 'Subansiri -']
----
Dima Hasao
['Haflong -', 'Mahur -', 'Maibong -', 'Umrangso -']
----
Nalbari
['Baganpara -', 'Banekuchi -', 'Barbhag -', 'Barkhetri -', 'Ghograpar -', 'Nalbari -', 'Pachim\nNalbari -', 'Tihu -']
----
Goalpara
['Balijana -', 'Dudhnoi -', 'Lakhipur -', 'Matia -', 'Rangjuli -']
----
Tamulpur
['Bajali -', 'Goreswar -', 'Pathorighat -', 'Rangia -', 'Tamulpur -']
----
Udalguri
['Dalgaon -', 'Dhekiajuli -', 'Harisinga -', 'Kalaigaon -', 'Khoirabari -', 'Mangaldoi -', 'Mazbat -', 'Pathorighat -', 'Udalguri -']
----
Kamrup
['Azara -', 'Boko -', 'Chamaria -', 'Chhaygaon -', 'Goroimari -', 'Hajo -', 'Jalah -', 'Kamalpur -', 'Kayan -',

/tmp/ipykernel_5155/993252146.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))


In [145]:
MASTER_DAMAGES = pd.concat(FILTERED_DAMAGES_dfs).reset_index(drop=True)

In [146]:
## Checking reveuue circle names
dc = 'Bongaigaon'
MASTER_DAMAGES[MASTER_DAMAGES.District==dc]


,Date,District,Number,Details,Detail_Length,Number of Damages (Date),Number of Damages (Long),NumberofDamages_RevenueCircle_level
87,2023-06-21,Bongaigaon,17,Manikpur - Dangaigaon kirtanpara Road | Dangai...,3834,17,17,"{'Manikpur ': 2, 'Bongaigaon ': 6, 'Srijangram..."
88,2023-06-25,Bongaigaon,22,Manikpur - NH-31 to Garugaon Road | Garugaon |...,4261,19,19,"{'Manikpur ': 14, 'Srijangram ': 5}"
89,2023-06-20,Bongaigaon,26,Bongaigaon - Pachim Bhadragaon to Digdari Road...,5559,26,26,"{'Bongaigaon ': 2, 'Boitamari ': 3, 'Manikpur ..."
90,2023-06-24,Bongaigaon,43,Bongaigaon - Kashidoba Kakoijana Road | Kashid...,9983,43,43,"{'Bongaigaon ': 9, 'Boitamari ': 6, 'Manikpur ..."


In [147]:
MASTER_DAMAGES[MASTER_DAMAGES.District==dc].NumberofDamages_RevenueCircle_level[89]

{'Bongaigaon ': 2, 'Boitamari ': 3, 'Manikpur ': 7, 'Srijangram ': 14}

In [148]:
14+3+5+2

24

In [149]:
road_damages_revcircle_df = pd.DataFrame(columns=['District','RevenueCircle','Number of damages','Date'])
i = 0
for idx, row in MASTER_DAMAGES.iterrows():
    damages_dict = row['NumberofDamages_RevenueCircle_level']
    for key in damages_dict.keys():
        rev_circle = key
        no_damages = damages_dict[key]
        
        road_damages_revcircle_df.loc[i,'RevenueCircle'] = rev_circle
        road_damages_revcircle_df.loc[i,'District'] = row['District']
        road_damages_revcircle_df.loc[i,'Number of damages'] = no_damages
        road_damages_revcircle_df.loc[i,'Date'] = row['Date']
        i=i+1
road_damages_revcircle_df.Date = pd.to_datetime(road_damages_revcircle_df.Date)

In [150]:
road_damages_revcircle_df.to_csv('Data_2023/Cleaned Data/RC_FRIMS_ROADS_DAMAGED_MASTER_2023.csv', index=False)